In [1]:
#run locally
import pandas as pd
import numpy as np
import os

anime = pd.read_csv("anime.csv")
users = pd.read_csv("rating.csv")
path="./"

<h2>Data Loading</h2>

In [ ]:
#run on kaggle environment
import kagglehub, os
import pandas as pd
import numpy as np

# Download latest version
path = kagglehub.dataset_download("CooperUnion/anime-recommendations-database")

print("Path to dataset files:", path)
os.listdir(path)

In [2]:
anime = pd.read_csv(os.path.join(path,"anime.csv"))
users = pd.read_csv(os.path.join(path,"rating.csv"))

<h2>Data Understanding</h2>

In [3]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


Dataset anime terdiri atas 7 variabel. 3 variabel merupakan variabel numerik (anime_id dan member memiliki tipe data int64 sedangkan rating memiliki tipedata float), sedangkan sisanya merupakan variabel kategorikal.

Menghitung jumlah data unik dalam dataset anime

In [4]:
anime.nunique()

anime_id    12294
name        12292
genre        3264
type            6
episodes      187
rating        598
members      6706
dtype: int64

In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


Data dalam variabel genre, nilai data merupakan sebuah list. Hal ini akan membuat ambiguitas dalam pemodelan nantinya. Data-data ini perlu di-encode agar proses cosine similarity dapat berjalan dengan baik. Namun, sebelum hal itu dilakukan, dataset perlu dicek terlebih dahulu apakah terdapat data yang tidak memiliki nilai.

terdapat data kosong dalam variable fitur type

In [6]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

terdapat 62 data kosong dalam variabel genre, 25 dalam variabel type dan 230  di rating.

<h3>Membersihkan data yang tidak memiliki nilai</h3>

In [7]:
anime=anime.dropna()
anime.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

Setelah data-data yang tidak memiliki nilai dihapus, dataset sekarang tidak memiliki nilai yang kosong. Dataset diurutkan berdasarkan anime_id.

In [8]:
anime=anime.sort_values("anime_id")
anime

,anime_id,name,genre,type,episodes,rating,members
22,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
152,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,8.40,137636
214,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
2095,7,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,7.36,64905
3159,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848
...,...,...,...,...,...,...,...
9991,34476,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51
10444,34490,Sushi Azarashi,Comedy,TV,30,3.00,12
9266,34503,Kochinpa! Dainiki,Comedy,TV,24,3.40,75
2726,34514,Pokemon Generations,"Action, Adventure, Fantasy, Game, Kids",ONA,18,7.21,295


In [9]:
anime.shape

(12017, 7)

Data yang telah dibersihkan sekarang memiliki 12017 baris data.

Memeriksa apakah data memiliki data duplikat

In [10]:
anime=anime.drop_duplicates()
anime

,anime_id,name,genre,type,episodes,rating,members
22,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
152,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,8.40,137636
214,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
2095,7,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,7.36,64905
3159,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848
...,...,...,...,...,...,...,...
9991,34476,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51
10444,34490,Sushi Azarashi,Comedy,TV,30,3.00,12
9266,34503,Kochinpa! Dainiki,Comedy,TV,24,3.40,75
2726,34514,Pokemon Generations,"Action, Adventure, Fantasy, Game, Kids",ONA,18,7.21,295


Dataset tetap memiliki 12017 baris data. Data tidak memiliki nilai-nilai duplikat.

<h3>Vektorisasi Genre</h3>
<p>Agar data dapat dilihat kemiripannya, perlu dilakukan cosine similarity yang berupa vektor. Namun karena data pada genre masih dalam bentuk list pada nilai datanya, hal ini akan menimbulkan ambiguitas dan model akan kesulitan untuk mengidentifikasi kemiripannya. Oleh karena itu nilai-nilai pada list perlu dipecah terlebih dahulu ke satu array/list yang terdiri atas data genre satuan. Setelah itu, dicocokan dengan genre apa saja yang ada dalam suatu data. Secara singkat, data-data genre yang unik dikumpulkan, dilakukan one-hot encoding dan memberikan nilai 1 kepada data yang terdapat dalam list genre.</p>

Mengumpulkan data-data genre yang unik ke dalam satu list

In [11]:
genres=[]
for genre_list in anime.genre:
    genre_list=genre_list.split(", ")
    for genre in genre_list:
        if genre not in genres:
            genres.append(genre)

Melakukan one-hot encoding berdasarkan list data-data unik genre yang sebelumnya dikumpulkan. Dataframe dibuat dan diberikan nilai 0 pada setiap data agar jika data terdapat di list genre, kolum genre tersebut akan bernilai 1.

In [12]:
genre_dict={}
for i in genres:
    genre_dict["genre_"+i]=[0]
genres=pd.DataFrame(genre_dict,dtype=int)
genres

,genre_Action,genre_Adventure,genre_Comedy,genre_Drama,genre_Sci-Fi,genre_Space,genre_Mystery,genre_Magic,genre_Police,genre_Supernatural,...,genre_Shoujo,genre_Harem,genre_Music,genre_Shoujo Ai,genre_Shounen Ai,genre_Kids,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p>Terdapat 43 variabel genre hasil one-hot encoding</p>
<p>Sekarang, menggabungkan dataframe genre yang telah dilakukan one-hot encoding dengan dataframe anime</p>

In [13]:
anime_joined=pd.concat([anime.anime_id,genres],axis=1)
anime_joined

,anime_id,genre_Action,genre_Adventure,genre_Comedy,genre_Drama,genre_Sci-Fi,genre_Space,genre_Mystery,genre_Magic,genre_Police,...,genre_Shoujo,genre_Harem,genre_Music,genre_Shoujo Ai,genre_Shounen Ai,genre_Kids,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi
22,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3159,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,34476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10444,34490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9266,34503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2726,34514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Karena penggabungan, 

In [14]:
anime_joined=anime_joined.fillna(int(0))
anime_joined=anime_joined.astype('int')
anime_joined

,anime_id,genre_Action,genre_Adventure,genre_Comedy,genre_Drama,genre_Sci-Fi,genre_Space,genre_Mystery,genre_Magic,genre_Police,...,genre_Shoujo,genre_Harem,genre_Music,genre_Shoujo Ai,genre_Shounen Ai,genre_Kids,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi
22,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2095,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3159,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,34476,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10444,34490,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9266,34503,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2726,34514,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
anime_joined.isnull().sum()

anime_id               0
genre_Action           0
genre_Adventure        0
genre_Comedy           0
genre_Drama            0
genre_Sci-Fi           0
genre_Space            0
genre_Mystery          0
genre_Magic            0
genre_Police           0
genre_Supernatural     0
genre_Fantasy          0
genre_Shounen          0
genre_Sports           0
genre_Josei            0
genre_Romance          0
genre_Slice of Life    0
genre_Cars             0
genre_Seinen           0
genre_Horror           0
genre_Psychological    0
genre_Thriller         0
genre_Martial Arts     0
genre_Super Power      0
genre_School           0
genre_Ecchi            0
genre_Vampire          0
genre_Historical       0
genre_Military         0
genre_Dementia         0
genre_Mecha            0
genre_Demons           0
genre_Samurai          0
genre_Game             0
genre_Shoujo           0
genre_Harem            0
genre_Music            0
genre_Shoujo Ai        0
genre_Shounen Ai       0
genre_Kids             0


In [16]:
anime_joined=pd.merge(anime,anime_joined,on="anime_id")
anime_joined

,anime_id,name,genre,type,episodes,rating,members,genre_Action,genre_Adventure,genre_Comedy,...,genre_Shoujo,genre_Harem,genre_Music,genre_Shoujo Ai,genre_Shounen Ai,genre_Kids,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,8.40,137636,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,7.36,64905,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12012,34476,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12013,34490,Sushi Azarashi,Comedy,TV,30,3.00,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12014,34503,Kochinpa! Dainiki,Comedy,TV,24,3.40,75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12015,34514,Pokemon Generations,"Action, Adventure, Fantasy, Game, Kids",ONA,18,7.21,295,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
for i,genres in enumerate(anime_joined.genre):
    for genre in genres.split(", "):
        label="genre_"+genre
        anime_joined.loc[i,label]=1

In [18]:
anime_joined

,anime_id,name,genre,type,episodes,rating,members,genre_Action,genre_Adventure,genre_Comedy,...,genre_Shoujo,genre_Harem,genre_Music,genre_Shoujo Ai,genre_Shounen Ai,genre_Kids,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,8.40,137636,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,7,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,7.36,64905,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12012,34476,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12013,34490,Sushi Azarashi,Comedy,TV,30,3.00,12,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12014,34503,Kochinpa! Dainiki,Comedy,TV,24,3.40,75,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12015,34514,Pokemon Generations,"Action, Adventure, Fantasy, Game, Kids",ONA,18,7.21,295,1,1,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
anime_joined=pd.concat([anime_joined,pd.get_dummies(anime_joined["type"],prefix="type",dtype=int)],axis=1)
anime_joined

,anime_id,name,genre,type,episodes,rating,members,genre_Action,genre_Adventure,genre_Comedy,...,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,8.40,137636,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,7,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,7.36,64905,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12012,34476,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12013,34490,Sushi Azarashi,Comedy,TV,30,3.00,12,0,0,1,...,0,0,0,0,0,0,0,0,0,1
12014,34503,Kochinpa! Dainiki,Comedy,TV,24,3.40,75,0,0,1,...,0,0,0,0,0,0,0,0,0,1
12015,34514,Pokemon Generations,"Action, Adventure, Fantasy, Game, Kids",ONA,18,7.21,295,1,1,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
genres_features=anime_joined.columns.to_list()[7:]
genres_features

['genre_Action',
 'genre_Adventure',
 'genre_Comedy',
 'genre_Drama',
 'genre_Sci-Fi',
 'genre_Space',
 'genre_Mystery',
 'genre_Magic',
 'genre_Police',
 'genre_Supernatural',
 'genre_Fantasy',
 'genre_Shounen',
 'genre_Sports',
 'genre_Josei',
 'genre_Romance',
 'genre_Slice of Life',
 'genre_Cars',
 'genre_Seinen',
 'genre_Horror',
 'genre_Psychological',
 'genre_Thriller',
 'genre_Martial Arts',
 'genre_Super Power',
 'genre_School',
 'genre_Ecchi',
 'genre_Vampire',
 'genre_Historical',
 'genre_Military',
 'genre_Dementia',
 'genre_Mecha',
 'genre_Demons',
 'genre_Samurai',
 'genre_Game',
 'genre_Shoujo',
 'genre_Harem',
 'genre_Music',
 'genre_Shoujo Ai',
 'genre_Shounen Ai',
 'genre_Kids',
 'genre_Hentai',
 'genre_Parody',
 'genre_Yuri',
 'genre_Yaoi',
 'type_Movie',
 'type_Music',
 'type_ONA',
 'type_OVA',
 'type_Special',
 'type_TV']

In [22]:
anime_joined[genres_features].to_numpy().shape

(12017, 49)

In [23]:
pd.DataFrame(anime_joined[genres_features].to_numpy(),index=anime_joined.name,columns=genres_features)

,genre_Action,genre_Adventure,genre_Comedy,genre_Drama,genre_Sci-Fi,genre_Space,genre_Mystery,genre_Magic,genre_Police,genre_Supernatural,...,genre_Hentai,genre_Parody,genre_Yuri,genre_Yaoi,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
name,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Cowboy Bebop: Tengoku no Tobira,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Trigun,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Witch Hunter Robin,1,0,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
Beet the Vandel Buster,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Platonic Chain: Ansatsu Jikkouchuu,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Sushi Azarashi,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Kochinpa! Dainiki,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(anime_joined[genres_features].to_numpy()) 

In [25]:
cosine_sim

array([[1.        , 0.6172134 , 0.75592895, ..., 0.53452248, 0.3086067 ,
        0.26726124],
       [0.6172134 , 1.        , 0.40824829, ..., 0.        , 0.16666667,
        0.        ],
       [0.75592895, 0.40824829, 1.        , ..., 0.70710678, 0.20412415,
        0.35355339],
       ...,
       [0.53452248, 0.        , 0.70710678, ..., 1.        , 0.        ,
        0.5       ],
       [0.3086067 , 0.16666667, 0.20412415, ..., 0.        , 1.        ,
        0.        ],
       [0.26726124, 0.        , 0.35355339, ..., 0.5       , 0.        ,
        1.        ]])

In [26]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=anime_joined.name, columns=anime_joined.name)
print('Shape:', cosine_sim_df.shape)

Shape: (12017, 12017)


In [27]:
cosine_sim_df

name,Cowboy Bebop,Cowboy Bebop: Tengoku no Tobira,Trigun,Witch Hunter Robin,Beet the Vandel Buster,Eyeshield 21,Hachimitsu to Clover,Hungry Heart: Wild Striker,Initial D Fourth Stage,Monster,...,Hashiri Hajimeta bakari no Kimi ni,Masou Gakuen HxH Specials,Uma Musume: Pretty Derby,Gijinka de Manabo!,Monster Strike: Rain of Memories,Platonic Chain: Ansatsu Jikkouchuu,Sushi Azarashi,Kochinpa! Dainiki,Pokemon Generations,Mobile Suit Gakuen: G-Reco Koushien
name,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,1.000000,0.617213,0.755929,0.428571,0.338062,0.507093,0.507093,0.338062,0.462910,0.267261,...,0.0,0.000000,0.267261,0.436436,0.188982,0.218218,0.534522,0.534522,0.308607,0.267261
Cowboy Bebop: Tengoku no Tobira,0.617213,1.000000,0.408248,0.462910,0.000000,0.182574,0.182574,0.000000,0.333333,0.288675,...,0.0,0.000000,0.288675,0.235702,0.204124,0.235702,0.000000,0.000000,0.166667,0.000000
Trigun,0.755929,0.408248,1.000000,0.377964,0.223607,0.670820,0.447214,0.447214,0.408248,0.176777,...,0.0,0.000000,0.353553,0.577350,0.250000,0.288675,0.707107,0.707107,0.204124,0.353553
Witch Hunter Robin,0.428571,0.462910,0.377964,1.000000,0.338062,0.338062,0.338062,0.169031,0.462910,0.534522,...,0.0,0.000000,0.267261,0.000000,0.188982,0.000000,0.267261,0.267261,0.154303,0.000000
Beet the Vandel Buster,0.338062,0.000000,0.223607,0.338062,1.000000,0.400000,0.200000,0.400000,0.182574,0.158114,...,0.0,0.000000,0.000000,0.000000,0.223607,0.000000,0.316228,0.316228,0.365148,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Platonic Chain: Ansatsu Jikkouchuu,0.218218,0.235702,0.288675,0.000000,0.000000,0.000000,0.000000,0.258199,0.000000,0.000000,...,0.0,0.333333,0.000000,0.333333,0.000000,1.000000,0.000000,0.000000,0.000000,0.408248
Sushi Azarashi,0.534522,0.000000,0.707107,0.267261,0.316228,0.632456,0.632456,0.632456,0.288675,0.250000,...,0.0,0.000000,0.000000,0.408248,0.000000,0.000000,1.000000,1.000000,0.000000,0.500000
Kochinpa! Dainiki,0.534522,0.000000,0.707107,0.267261,0.316228,0.632456,0.632456,0.632456,0.288675,0.250000,...,0.0,0.000000,0.000000,0.408248,0.000000,0.000000,1.000000,1.000000,0.000000,0.500000


In [28]:
def anime_recommendations(anime_title, similarity_data=cosine_sim_df, items=anime_joined[['name','type','genre']], k=5):
    """
    Rekomendasi anime berdasarkan kemiripan dataframe
 
    Parameter:
    ---
    anime_title : tipe data string (str)
                judul anime (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan resto sebagai 
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---
 
 
    Pada index ini, kita mengambil k dengan nilai similarity terbesar 
    pada index matrix yang diberikan (i).
    """
 
 
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan    
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,anime_title].to_numpy().argpartition(kth=-k)
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop anime_title agar nama resto yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(anime_title, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

In [32]:
anime_recommendations("Toaru Majutsu no Index")

,name,type,genre
0,Persona: Trinity Soul,TV,"Action, Sci-Fi, Super Power"
1,Toaru Majutsu no Index II,TV,"Action, Magic, Sci-Fi, Super Power"
2,Toaru Kagaku no Railgun,TV,"Action, Sci-Fi, Super Power"
3,Toaru Kagaku no Railgun S,TV,"Action, Sci-Fi, Super Power"
4,Ougon Bat,TV,"Action, Sci-Fi, Super Power"


<h2>Collaborative Filtering</h2>

In [34]:
import pandas as pd
import numpy as np 
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

<h3>Data Understanding</h3>

In [35]:
df = users

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [37]:
df

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


rating -1 berarti penonton hanya menonton dan tidak memberikan rating

In [38]:
df = df.loc[df["rating"]!=-1]
df

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


<h3>Data Preparation</h3>
Mengecek apakah dataset memiliki nilai kosong

In [39]:
df.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

Dataset tidak memiliki nilai data kosong

<h4>Encoding user data</h4>
UserId and animeId perlu disandikan (encode) agar mudah terbaca oleh model

In [40]:
userId = df.user_id.unique().tolist()
animeId = df.anime_id.unique().tolist()

#encoding userId
userIdEncoded = {x:i for i,x in enumerate(userId)}
# print("encoded userId: ",userIdEncoded)

#encoding angka ke userId
encodedToUserId = {i:x for i,x in enumerate(userId)}
# print("encoded angka ke userId: ",encodedToUserId)

#encoding animeId
animeIdEncoded = {x:i for i,x in enumerate(animeId)}
# print("encoded animeid: ",animeIdEncoded)

#encoding angka ke animeId
encodedToAnimeId = {i:x for i,x in enumerate(animeId)}
# print("encoded angka ke animeId: ",encodedToAnimeId)


Mapping user_id dan anime_id ke dataframe

In [41]:
df.user_id = df.user_id.map(userIdEncoded)
df.anime_id = df.anime_id.map(animeIdEncoded)
df

/tmp/ipykernel_30/2278938822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.user_id = df.user_id.map(userIdEncoded)
/tmp/ipykernel_30/2278938822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.anime_id = df.anime_id.map(animeIdEncoded)


,user_id,anime_id,rating
47,0,0,10
81,0,1,10
83,0,2,10
101,0,3,10
153,1,4,10
...,...,...,...
7813732,69598,56,7
7813733,69598,1802,9
7813734,69598,783,10
7813735,69599,753,9


In [42]:
# Mendapatkan jumlah user
num_users = len(userIdEncoded)
print(num_users)
 
# Mendapatkan jumlah judul anime
num_anime = len(animeIdEncoded)
print(num_anime)
 
# Mengubah rating menjadi nilai float
df['rating'] = df['rating'].values.astype(np.float32)
 
# Nilai minimum rating
min_rating = min(df['rating'])
 
# Nilai maksimal rating
max_rating = max(df['rating'])
 
print('Number of User: {}, Number of Resto: {}, Min Rating: {}, Max Rating: {}'.format(
    num_users, num_anime, min_rating, max_rating
))

/tmp/ipykernel_30/1688598679.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].values.astype(np.float32)


69600
9927
Number of User: 69600, Number of Resto: 9927, Min Rating: 1.0, Max Rating: 10.0


Mempersiapkan data training dan test

In [43]:
from sklearn.model_selection import train_test_split

#mengacak data
df = df.sample(frac=1, random_state=42)

x = df[["user_id","anime_id"]].values
y = df["rating"].apply(lambda x: (x - min_rating)/(max_rating - min_rating)).values

X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.2, random_state=42)
X_train

array([[51195,    66],
       [40544,   615],
       [47709,  2927],
       ...,
       [57576,  1484],
       [15379,   243],
       [ 6953,  2691]])

<h2>Training Model</h2>

In [44]:
class RecommenderNet(tf.keras.Model):
 
  # Insialisasi fungsi
  def __init__(self, num_users, num_anime, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_anime = num_anime
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.anime_embeddings = layers.Embedding( # layer embeddings anime
        num_anime,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.anime_bias = layers.Embedding(num_anime, 1) # layer embedding anime bias
 
  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) # memanggil layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) # memanggil layer embedding 2
    anime_vector = self.anime_embeddings(inputs[:, 1]) # memanggil layer embedding 3
    anime_bias = self.anime_bias(inputs[:, 1]) # memanggil layer embedding 4
 
    dot_user_resto = tf.tensordot(user_vector, anime_vector, 2) 
 
    x = dot_user_resto + user_bias + anime_bias
    
    return tf.nn.sigmoid(x) # activation sigmoid

In [45]:
model = RecommenderNet(num_users, num_anime, 50) # inisialisasi model
 
# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [46]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get("root_mean_squared_error")<=0.15):
      print("root_mean_squared_error telah mencapai <=0.15!")
      self.model.stop_training = True
callbacks = myCallback()

reduce_lr = ReduceLROnPlateau(monitor='root_mean_squared_error', factor=0.5, patience=5, min_lr=1e-4, verbose=1)
early_stopping = EarlyStopping(monitor="root_mean_squared_error", mode='min', min_delta=0.1, patience=10, restore_best_weights=True)

In [ ]:
# Memulai training
 
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    epochs = 100,
    validation_data = (X_test, y_test),callbacks=[callbacks, reduce_lr, early_stopping])

Epoch 1/100


I0000 00:00:1733431237.241270      79 service.cc:145] XLA service 0x79f00400af10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733431237.241324      79 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733431237.241346      79 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


   83/79216 ━━━━━━━━━━━━━━━━━━━━ 2:27 2ms/step - loss: 0.6932 - root_mean_squared_error: 0.3137 

I0000 00:00:1733431238.374073      79 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


79216/79216 ━━━━━━━━━━━━━━━━━━━━ 469s 6ms/step - loss: 0.5699 - root_mean_squared_error: 0.1884 - val_loss: 0.5372 - val_root_mean_squared_error: 0.1477 - learning_rate: 0.0010
Epoch 2/100
79210/79216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5339 - root_mean_squared_error: 0.1442

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model_metrics')
plt.ylabel('root_mean_squared_error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<h3>Menyimpan Model</h3>

In [ ]:
model.save('my_model.keras')
